# Human Resources Department Attrition Analysis
<span style=color:gray> **In this analysis, we explore the data given to us to help understand the current attrition rate of the company. Certain factors like (Age, Gender, Monthly Income, Over Time Work and a few more) will be looked at to help determine what the main cause of turnover for employees. At the end, we hope to have analyzed the situation and dataset thoroughly and be able to submit the best recommendation to remedy the concern.** </span>

<span style=color:green> <font size="4"> **The code below shows how to load the appropriate libraries where packages can be pulled from and create data.** </span> </font>

In [ ]:
shh <- suppressPackageStartupMessages
shh(library(ggplot2))
shh(library(caret))
shh(library(rpart))
shh(library("randomForest"))
install.packages('ggcorrplot')
shh (library(ggcorrplot))
shh (library(pROC))
install.packages('pROC')

# Set standard dimensions for plots.
options(repr.plot.width=4, repr.plot.height=3)

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


<span style=color:green> <font size="4"> **This code allows us to import data from a csv file where all data are coming from.** </span> </font>

In [ ]:
hr_data <- read.csv('https://bit.ly/2VTrfPs')

<font size="5"> **The next four data sets you will see allows you to gain basic knowledge about the the Company's Attrition Rate and the variables that dictate it.** </font>

<span style=color:green> <font size="4"> **The code below calculates the dimensions of the dataset.** </span> </font>

In [ ]:
dim(hr_data)

<span style=color:green> <font size="4"> **Preview of the first few records in the dataset.** </span> </font>

In [ ]:
head(hr_data)

<span style=color:green> <font size="4"> **List and describe each of the fields in the dataset.** </span> </font>

In [ ]:
str(hr_data)

<span style=color:green> <font size="4"> **Calculated descriptive stats for each of the fields in the dataset.** </span> </font>

In [ ]:
summary(hr_data)

<span style=color:green> <font size="5"> **As we explore the possible reasons for the cause of Attrition, I  have lister five questions below that can help evaluate that data we have and see which factors cause employees to transfer work.** </span> </font>

<span style=color:red> <font size="4"> **1. Does one gender tend to quit more frequently than the other?** </font> </span>

In [ ]:
ggplot(hr_data,aes(x=Gender , fill = factor(Attrition))) + 
    geom_bar(position= "fill") + xlab("Gender") + ylab("Attrition") +
scale_fill_manual(values = c("grey", "red"), name = "Turnover Rate", labels = c("Stayed", "Left"))

<span style=color:green> <font size="4"> The bar chart above depicts the number of turnover the company had within each gender against how much emlpoyees they have.
More male employees seem to churn and transfer work than female ones. </span> </font>

In [ ]:
ggplot(hr_data, aes(x=Gender, y=Attrition)) + 
    geom_line() + (aes(color = Gender))

<span style=color:green> <font size="4"> An additional line chart was made to emphasize the situation. </span> </font>

<span style=color:red> <font size="4"> **2. Does age appear to make a difference? Are our younger workers less committed to our company?** </font> </span>

In [ ]:
ggplot(hr_data,aes(x=Age, fill = factor(Attrition))) +
    geom_density(position= "fill") + scale_fill_manual(values = c("yellow", "green"), name = "Turnover Rate", labels = c("Stayed", "Left"))

<span style=color:green> <font size="4"> From the density chart above, it looks like that the younger generation (millennials and generation Z) have the tendency to transfer work more than the older generation. </span> </font>

<span style=color:red> <font size="4"> **3. Are we working our employees too hard? Is our overtime requirement for some employees driving them away?** </font> </span>

In [ ]:
ggplot(hr_data,aes(OverTime,fill=factor(Attrition))) +
    geom_bar() + xlab("OverTime") + ylab("Attrition") +
scale_fill_manual(values = c("lightblue", "purple"), name = "Turnover Rate", labels = c("Stayed", "Left"))

<span style=color:green> <font size="4"> The bar chart shows you that even though there are fewer people who does overtime for the company, the turnover rate is still higher for them, compared to the people who don't do overtime. </span> </font>

In [ ]:
ggplot(hr_data, aes(x=OverTime, y=Attrition)) + 
    geom_line() + (aes(color = OverTime)) +
scale_fill_manual(values = c("lightblue", "purple"), name = "Turnover Rate", labels = c("Stayed", "Left"))

<span style=color:red> <font size="4"> **4. Our concern is that the highest performing employees are leaving at a higher rate. Is that true?** </font> </span>

In [ ]:
ggplot(hr_data,aes(Attrition,PerformanceRating)) + geom_jitter()+ ylab("PerformanceRating")+ xlab("Attrition")

<span style=color:green> <font size="4"> According to the scatterplot above, you'll be happy to know that although there is a attrition in the company, we are able to retain people who have a high performance rating for the most part. </span> </font>

<span style=color:red> <font size="4"> **5. Is this about money? Are we not paying employees enough? Were the raises not strong enough? Is that why they leave?** </font> </span>

In [ ]:
ggplot(hr_data, aes(x = MonthlyIncome, fill = Attrition)) + 
    geom_dotplot() + (aes(color = Gender))

<span style=color:green> <font size="4"> You will see from the dotplot above that the employees who receive lower wages tend to leave the company more so than the ones with a higher pay. </span> </font>

<span style=color:red> <font size="5"> **The next data sets are an attempt to predict and split data to come to a more accurate result for the best recommendation.** </span> </font>

<span style=color:green> <font size="4"> **This code below evaluates whether or not there are missing/null values and addresses those, if they exist.** </span> </font>

In [ ]:
sum(is.na(hr_data))

In [ ]:
cor <- cor(hr_data[c(1,3,8,12:25)])

options(repr.plot.width=7, repr.plot.height=5)
ggcorrplot(cor)

<span style=color:green> <font size="4"> **This code splits the data into training and test sets (using a set seed of "3456".** </span> </font>

In [ ]:
# Allows to set the random number generator to the same.
set.seed(3456)  
trainIndex <- createDataPartition(hr_data$Attrition, p = .8, list = FALSE, times = 1)
hr_train <- hr_data[ trainIndex,] 
hr_test <- hr_data[-trainIndex,]

<span style=color:green> <font size="4"> **This code uses logistic regression to create your prediction model.** </span> </font>

In [ ]:
# Try to build a regression model with all variables.
hr_train_model <- glm (Attrition ~ Age + Gender + MaritalStatus + MonthlyIncome, 
                          data=hr_train, family=binomial)

summary(hr_train_model)

<span style=color:green> <font size="4"> **This code creates a decision tree to help calculate the prediction model.** </span> </font>

In [ ]:
hr_train_model <- rpart(Attrition~ Age + Gender + MonthlyIncome + OverTime + Department,
        method="class", data=hr_data)

print(hr_train_model)
plotcp(hr_train_model)

# create additional plots
par(mfrow=c(1,2)) # two plots on one page

# plot tree
plot(hr_train_model, uniform=TRUE,
   main="Attrition Rate")
text(hr_train_model, use.n=TRUE, all=TRUE, cex=.2)

<span style=color:green> <font size="4"> **This code assess the importance of each variable.** </span> </font>

In [ ]:
varImp(hr_train_model)

In [ ]:
# Evaluate variable importance
varImp(hr_train_model, scale=FALSE)

importance <- as.data.frame(varImp(hr_train_model))
importance <- data.frame(Importance = immportnace$Overall,
                        Fiel = rownames(importance))
importance[order(-importance$Importance),]


<span style=color:green> <font size="4"> **This code adds and populates the "Attrition" prediction in the test dataset.** </span> </font>

In [ ]:
hr_test$Attrition_prediction <- predict(hr_train_model,newdata=hr_test,type='response')
head(hr_test)

<span style=color:green> <font size="4"> **This code evaluates the performance of each method/model using the area under the ROC curve.** </span> </font>

In [ ]:
ROC <- roc(hr_test$Attrition, hr_test$Attrition_prediction)

plot(ROC)

auc(ROC)

<span style=color:green> <font size="4"> **This code writes the "Attrition" predictions to a new file that can be downloaded and used for study and review.** </span> </font>

In [ ]:
hr_test_abbreviated <- hr_test[,c(1,2,26)]
head(hr_test_abbreviated)
write.csv(hr_test, "Attrition_test_predictions1.csv", row.names=FALSE)

<span style=color:red> <font size="6"> **RECOMMENDATION FOR MINIMIZING CURRENT RATE OF ATTRITION** </span> </font>

# Attrition Rate Hypothesis

<span style=color:black> After reviewing and analyzing the given data above, factors that contribute to a company's employee decision of leaving are: low-income pay, possible workload overload, a classification for the younger generation and low-performance ratings. Dissatisfaction over job responsibilities, salary or even work-environment can be major variables that can push one out the door. Especially for your millennials who the The Wall Street Journal described as a generation who seeks the "biggest gains in Happiness with Pay." </span>

Link for Article: https://l.facebook.com/l.php?u=https%3A%2F%2Fwww.wsj.com%2Farticles%2Fyounger-workers-report-biggest-gains-in-happiness-with-pay-11567071000%3Ffbclid%3DIwAR1OviZuLCtKBXJ5x3dnlNVTzxtQBj_hPmb2EPhIsJ5316mOunx_sG5DdwA&h=AT0veMbvwpBGMxp_cbaqGtp2dN2dNEvI93p5Jnd2rRApQclJrsgdvT5MWJSVRDopM6QK8d3rXJPx1PHUMmZjC7NRIHVsIpbGK4tNTjItfwxdWz3NI0MgSt2WgzrTBXBsgd-bTg

# Recommendation

 My personal recommendation is to revamp the compensation and benefits package of the company to a more competitive stand point if it can withstand the costs. According to Maslow's Hierarchy of Needs quotes by simplypsychology.org, "Our most basic need is for physical survival, and this will be the first thing that motivates our behavior. Once that level is fulfilled the next level up is what motivates us, and so on." People seek to increase their abilities to provide for their family's basic needs. Another recommendation is to create campaign or activities within the company that can help foster employee engagement and help them be engaged in the workplace and in their duties. This will decrease the possibility of burning out and will increase motivation to do better everyday.